In [3]:
import os 
from dotenv import load_dotenv


load_dotenv()
from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")

llm= ChatGroq(model_name="llama-3.3-70b-versatile",groq_api_key=groq_api_key)
llm
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings= HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
import bs4

In [12]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content","post-title","post-header")
        )
    ),
)

docs=loader.load()


In [14]:
text_splitter= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits= text_splitter.split_documents(docs)
vectorstore=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002226EDCBF80>, search_kwargs={})

In [15]:
system_prompt= (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved information to answer the user's questions."
    "If you dont know the answer, you can say 'I dont know'"
    "Use three sentences maximum and keep the answer concise"
    "\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [16]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)

rag_chain=create_retrieval_chain(retriever,question_answer_chain)


In [19]:
response=rag_chain.invoke({"input":"What is Self-Reflection?"})
response

{'input': 'What is Self-Reflection?',
 'context': [Document(id='98e58515-de5c-4235-ad6f-0a993bd07504', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 3. Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used as context for querying LLM.'),
  Document(id='c4a33830-fa47-4709-bba9-21ad77258fc8', metadata={'source': 'https://lilianweng.

In [20]:
response['answer']

'Self-reflection is a vital aspect that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It involves showing two-shot examples to a large language model (LLM) to guide future changes in the plan. This process enables agents to learn from their experiences and adapt to new situations.'

In [35]:
## Adding  Chat History

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt=(
    "Given a chat history and the lastest user question"
    "which might reference context in the chat history"
    "Answer the questions based on context in the chat history."
    "Do NOT answer the question,"
    "just reformulate it if needed and otherwise return it as is."

)

contextualize_q_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)


In [36]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002226EDCBF80>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [37]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}\nContext: {context}"),
    ]
)


In [38]:
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [39]:
from langchain_core.messages import AIMessage, HumanMessage
chat_history=[]
question="Explain Self-Reflection?"
response1= rag_chain.invoke({"input":question,"chat_history":chat_history})
chat_history.extend([HumanMessage(question),AIMessage(response1['answer'])])

question2="Tell me more about it"
response2=rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

What is the role of self-reflection in the Reflexion framework, and how does it enable autonomous agents to refine past action decisions, correct previous mistakes, and improve their reasoning skills in knowledge-intensive and decision-making tasks?


In [40]:
chat_history

[HumanMessage(content='Explain Self-Reflection?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='What is the concept of self-reflection in the context of autonomous agents, and how does it enable them to refine past action decisions and correct previous mistakes, particularly in relation to the Reflexion framework and its application in knowledge-intensive tasks and decision-making tasks?', additional_kwargs={}, response_metadata={})]

In [41]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store ={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

conversational_rag_chain=RunnableWithMessageHistory(
    rag_chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [42]:
conversational_rag_chain.invoke(
    {"input":"What is Task Decomonposition?"},
    config={
        "configurable":{"session_id":"abc123"}
    },
)["answer"]

'What is Task Decomposition in the context of LLM-powered autonomous agent systems, where it involves breaking down complex tasks into smaller, manageable steps, and can be achieved through various methods such as simple prompting, task-specific instructions, or human inputs?'